<a href="https://colab.research.google.com/github/rishav197/CSE508_Winter2024/blob/main/CSE508_Winter2024_A2_2020569/IR_A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install tensorflow

In [ ]:
import os
import pandas as pd
import requests
import cv2
import numpy as np
import nltk
import tensorflow as tf
from tensorflow import keras
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from urllib.parse import urlparse
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
import pickle
from collections import Counter
import math
from sklearn.preprocessing import normalize

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string

In [ ]:
# Initialize NLTK components
stop_words = set(stopwords.words('english'))
porter = PorterStemmer()
lemmatizer = WordNetLemmatizer()

Loading the given dataset

In [ ]:
# data = pd.read_csv("/content/drive/MyDrive/A2_Data.csv")
data = pd.read_csv("/content/drive/MyDrive/A2_Data.csv", skiprows=1, header=None, names=['ID', 'Image', 'Review Text'])
data.rename(columns={"Unnamed: 0":"ID"}, inplace=True)
data

,ID,Image,Review Text
0,3452,['https://images-na.ssl-images-amazon.com/imag...,Loving these vintage springs on my vintage str...
1,1205,['https://images-na.ssl-images-amazon.com/imag...,Works great as a guitar bench mat. Not rugged ...
2,1708,['https://images-na.ssl-images-amazon.com/imag...,We use these for everything from our acoustic ...
3,2078,['https://images-na.ssl-images-amazon.com/imag...,Great price and good quality. It didn't quite...
4,801,['https://images-na.ssl-images-amazon.com/imag...,I bought this bass to split time as my primary...
...,...,...,...
995,1265,['https://images-na.ssl-images-amazon.com/imag...,Extremely impressed with this kit.
996,1882,['https://images-na.ssl-images-amazon.com/imag...,This is a great stereo reverb with plenty of c...
997,1547,['https://images-na.ssl-images-amazon.com/imag...,I really like the simplicity of this bridge. I...
998,1004,['https://images-na.ssl-images-amazon.com/imag...,"Great Product, but there is no warranty in the..."


In [ ]:
data.columns

Index(['ID', 'Image', 'Review Text'], dtype='object')

**part-1: Image Feature Extraction**

Resnet is used as a pretrained model

In [ ]:
resnet = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

94765736/94765736 [==============================] - 3s 0us/step


In [ ]:
def preprocess_img(img):
  if img is None:
    print("Error: Empty image")
    return None

  # Resize the image to 224x224
  resized_img = cv2.resize(img, (224,224))

  img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB)

  #apply rotation
  if np.random.rand()>0.5:
    img = cv2.flip(img, 1)

  brightness = np.random.uniform(0.5, 2.0)
  exposure = np.random.uniform(0.5, 2.0)
  img = cv2.convertScaleAbs(img, alpha=brightness, beta=0)
  img = cv2.convertScaleAbs(img, alpha=exposure, beta=0)

  #Normalize pixel vals of the image to the range from 0 to 1
  img = img/255.0

  return img

In [ ]:
# method for extract features from images using ResNet50
def extract_features(img):
  x = np.expand_dims(img, axis=0)
  x = preprocess_input(x)
  features = resnet.predict(x)
  return features.flatten()

In [ ]:
image_data = dict()

In [ ]:

for idx, row in data.iterrows():
  prod_id = row["ID"]
  # print(type(prod_id), prod_id)
  img_urls = row["Image"]
  # print(img_urls)

  lst_of_features = []

  for idx, url in enumerate(eval(img_urls)):
    img_data = requests.get(url).content
    #Decode the image
    img = cv2.imdecode(np.frombuffer(img_data, np.uint8), -1)

    if img is None:
      print("Skipping invalid image: {}".format(url))
      continue

    #preprocess the image
    img = preprocess_img(img)

    if img is None:
      print("Skipping invalid image after preprocessing : {}".format(url))
      continue

    img_features = extract_features(img)

    lst_of_features.append({
        "image_features":img_features,
        "image_url":url
    })



  if lst_of_features:
    image_data[prod_id] = lst_of_features


In [ ]:
len(image_data)

In [ ]:
with open('image_data.pkl', 'wb') as fptr:
  pickle.dump(image_data, fptr)

In [ ]:
with open('/content/image_data.pkl', 'rb') as fptr:
  image_data = pickle.load(fptr)

In [ ]:
# image_data

**part-2: Text Feature Extraction**

a)

In [ ]:
def preprocess_txt(text):
  if type(text)==str:
    #converting the text into lower-case
    txt = text.lower()
    # print(txt)

    #tokenization
    tokens = word_tokenize(text)
    # print(tokens)

    #Removing puncts and stop words from tokens
    processed_tokens = [word for word in tokens if word not in string.punctuation and word not in stop_words]
    # print(processed_tokens)

    #apply stemming
    processed_tokens = [porter.stem(token) for token in processed_tokens]
    # print(processed_tokens)

    #apply lemmatization
    processed_tokens = [lemmatizer.lemmatize(token) for token in processed_tokens]
    # print(processed_tokens)

    if processed_tokens:
      #here we joining the processed tokens
      processed_txt = " ".join(processed_tokens)
      return processed_txt
    else:
      return "invalid text"

  else:
    return "invalid text"


In [ ]:
ans = preprocess_txt("This is a sample text for pre-processing.")
ans

b)

In [ ]:
#step-1: Calculating the term frequency (tf)
def calc_tf(doc):
  words = doc.split()
  word_cnt = Counter(words)

  tf = {}
  for word in words:
    val = word_cnt[word]/len(words)
    tf[word]=val

  return tf

#step-2: Calculating the Inverse Document Fequency (tf-idf)
def calc_idf(docs):
  N = len(docs)
  idf = {}
  lst_of_words = []

  for doc in docs:
    for word in set(doc.split()):
      lst_of_words.append(word)

  print(lst_of_words)

  for word in set(lst_of_words):
    word_cnt = 0
    for doc in docs:
      if word in doc:
        word_cnt+=1
    idf[word] = math.log(N / (1+word_cnt))

  return idf


#Step-3: Calculate the TF-IDF score
def calc_tfidf(docs):
  tfidf = []
  idf = calc_idf(docs)

  for doc in docs:
    tf = calc_tf(doc)
    for word in tf:
      tmp = {}
      tmp[word] = tf[word]*idf[word]
      tfidf.append(tmp)
  return tfidf


#Calculate the cosine similarity b/w two feature vectors
def calc_cosine_similarity(f1, f2):
  #Normalize feature vectors
  f1_normalized = normalize(f1.reshape(1, -1))
  f2_normalized = normalize(f2.reshape(1, -1))

  if f1_normalized.shape[1] > f2_normalized.shape[1]:
    f2_normalized = f2_normalized.T
  else:
    f1_normalized = f1_normalized.T

  #Here's we calculating cosine similarity
  return np.dot(f1_normalized, f2_normalized)
calc_tf("thi sampl text pre-process")

In [ ]:
#documents
docs = []

In [ ]:
for idx, row in data.iterrows():
  # print(idx)
  # print(row)

  reivew_txt = row["Review Text"]
  # print(reivew_txt)

  processed_review = preprocess_txt(reivew_txt)
  # print(processed_review)

  docs.append(processed_review)


In [ ]:
print(len(docs), docs)

In [ ]:
#calculate the TF-IDF vectors for the documents
tfidf_docs = calc_tfidf(docs)

In [ ]:
tfidf_data = {}

for idx, row in data.iterrows():
  review_txt = row["Review Text"]
  prod_id = row["ID"]

  lst_of_features = []
  lst_of_features.append({"review_txt":review_txt, "tfidf_score":tfidf_docs[idx]})
  tfidf_data[prod_id] = lst_of_features

In [ ]:
print(tfidf_data)

Save extracted features and the TF-IDF score

In [ ]:
with open('tfidf_data.pkl', 'wb') as fptr:
  pickle.dump(tfidf_data, fptr)

In [ ]:
with open('/content/tfidf_data.pkl', 'rb') as f:
  tfidf_data = pickle.load(f)

In [ ]:
tfidf_data

**part-3: Image and Text Retrieval**

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def get_cs(v1, v2):
  #apply padding on the smaller vector

  diff = abs(len(v1) - len(v2))
  if len(v1) > len(v2): #when v1 is greater than v2
    v2 = np.pad(v2, (0, diff), "constant")
  else: #when v2 is greater than v1
    v1 = np.pad(v1, (0, diff), "constant")

  #Reshape v1 and v2
  v1 = v1.reshape(1, -1)
  v2 = v2.reshape(1, -1)

  #calculate the cosine similarity
  return cosine_similarity(v1, v2)

In [ ]:
input_image_url = 'https://images-na.ssl-images-amazon.com/images/I/71bztfqdg+L._SY88.jpg'
input_review_text = 'I have been using Fender locking tuners for about five years on various strats and teles. Definitely helps with tuning stability and way faster to restring if there is a break.'

In [ ]:
def get_img_features_from_url(url):
  img_data = requests.get(url).content
  img = cv2.imdecode(np.frombuffer(img_data, np.uint8), -1)
  img_features = extract_features(img)

  return img_features

In [ ]:
# Extract features from the input image using VGG16 (you need to implement this part)
input_image_features = get_img_features_from_url(input_image_url)

In [ ]:
processed_review_txt = preprocess_txt(input_review_text)
print(processed_review_txt)

input_tfidf_score = calc_tfidf([processed_review_txt])
print(input_tfidf_score)

Top3 three similar images

In [ ]:
lst_of_cs_img = [] #list of cosine similarities images

for prod_id, images in image_data.items():
  # print(prod_id)
  # print(len(images), images)

  for image in images:
    score = get_cs(input_image_features, image["image_features"])
    lst_of_cs_img.append((prod_id, image["image_url"], score))


print(lst_of_cs_img)

In [ ]:
lst_of_cs_img.sort(key=lambda x: x[2], reverse=True)

print(lst_of_cs_img)

#top three docs with higher cosine similarity
top3_similar_imgs = lst_of_cs_img[:3]

In [ ]:
top3_similar_imgs

Top3 three similar texts

In [ ]:
lst_of_cs_txt = [] #list of cosine similarities texts


for prod_id, txt in tfidf_data.items():
  # print(prod_id, txt)
  for review_txt in txt:
    score = get_cs(np.array(list(input_tfidf_score[0].values())), np.array(list(review_txt["tfidf_score"].values())))
    lst_of_cs_txt.append((prod_id, review_txt["review_txt"], score))

print(lst_of_cs_txt)

In [ ]:
lst_of_cs_txt.sort(key=lambda x: x[2], reverse=True)
print(lst_of_cs_txt)

top3_similar_txts = lst_of_cs_txt[:3]
print(top3_similar_txts)

In [ ]:
top3_similar_txts

Composite Similarity Score

In [ ]:
lst_of_composite_similarities = {}

for e1 in lst_of_cs_txt:
  # print(e1)
  lst_of_docs = []
  for e2 in lst_of_cs_img:
    if e1[0]==e2[0]:
      lst_of_docs.append({"image_url":e2[1], "review_txt":e1[1], "combine_score":(e1[2][0][0]+e2[2][0][0])/2})
  lst_of_composite_similarities[e1[0]]=lst_of_docs

In [ ]:
lst_of_composite_similarities

In [ ]:
lst_of_composite_similarities[170][0]["combine_score"]

Image Retrieval

In [ ]:
!pip install quo

In [ ]:
from quo import echo

In [ ]:
print("-"*80)
echo("{}".format("USING IMAGE RETRIEVAL"), underline=True)



for idx in range(len(top3_similar_imgs)):
  # print(top3_similar_imgs[idx])
  img = top3_similar_imgs[idx]
  # print(img)
  print("{}) \033[1mImage URL:\033[0m {}".format(idx+1, img[1]))
  for review in tfidf_data[img[0]]:
    print("   \033[1mReview:\033[0m {}".format(review["review_txt"]))

  print("   Cosine Similarity of images: {}".format(img[2][0][0]))

  for idx2 in range(len(lst_of_cs_txt)):
    if lst_of_cs_txt[idx2][0] == img[0]:
      print("   Cosine Similarity of text: {}".format(lst_of_cs_txt[idx2][2][0][0]))
      break
  print("   Composite Similarity score: {}".format(lst_of_composite_similarities[img[0]][0]["combine_score"]))
  print()

Text Retrieval

In [ ]:
print("-"*80)
echo("{}".format("USING TEXT RETRIEVAL"), underline=True)


for idx in range(len(top3_similar_txts)):
  txt = top3_similar_txts[idx]
  # print(txt)
  print("{}) \033[1mImage URL:\033[0m".format(idx+1), end="")
  print("[", end="")
  count=1
  for img in image_data[txt[0]]:
    print(img["image_url"], end="")
    if count<len(image_data[txt[0]]):
      print(", ", end="")
    count+=1
  print("]")
  print("   \033[1mReview:\033[0m {}".format(txt[1]))
  for idx2 in range(len(lst_of_cs_img)):
    if lst_of_cs_img[idx2][0]==txt[0]:
      print("   Cosine Similarity of images: {}".format(lst_of_cs_img[idx2][2][0][0]))
      break
  print("   Cosine Similarity of text: {}".format(txt[2][0][0]))
  print("   Composite Similarity score: {}".format(lst_of_composite_similarities[txt[0]][0]["combine_score"]))
  print()